# Database Archive

## Execution

### Import packages

In [11]:
%matplotlib widget

import os

from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

### Data Visualization 

In [12]:
from utility import db_conn
import config 

username = os.environ.get("db_user")
password = os.environ.get("db_pwd")
ori_db_name_button = widgets.RadioButtons(
        options=['reformer', 'Reformer_BW', 'Reformer_SE'],
        description='From Database:',
        disabled=False
        )
cur = db_conn(username=username, password=password, db_name=ori_db_name_button.value)
cur.execute('SHOW tables')

table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Select Table')

arch_db_name_button = widgets.RadioButtons(
        options=['reformer', 'Reformer_BW', 'Reformer_SE'],
        description='Archive into:',
        disabled=False
        )

toggle = widgets.ToggleButton(value=False, description='Exec', disabled=False, button_style='success', tooltip='Description',icon='check')
tb_name = widgets.Text(
    value=f'{table_dropdown.value}',
    placeholder='Type Table Name',
    description='New Name:',
    disabled=False
)

def update_db_name(change):
    db_name = change.new
    cur = db_conn(username=username, password=password, db_name=change.new)
    cur.execute('SHOW tables')
    table_dropdown.options = [u for i in cur.fetchall()[:] for u in i][::-1]
ori_db_name_button.observe(update_db_name, 'value') 

def exec(change):
    if change.new:
        cur = db_conn(username=username, password=password, db_name=arch_db_name_button.value)
        display(f"create table {tb_name.value} as select * from {ori_db_name_button.value}.{table_dropdown.value};")
        cur.execute(f"create table {tb_name.value} as select * from {ori_db_name_button.value}.{table_dropdown.value};")
        display('Succeed')
    else:
        pass
toggle.observe(exec, 'value') 

widgets.VBox([ori_db_name_button, table_dropdown, arch_db_name_button, tb_name, toggle])

### Close Remote Connection 

In [10]:
conn.close()

NameError: name 'conn' is not defined